# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [18]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [19]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [20]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [24]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    sfo_year_data = sfo_data.groupby('year')
    housing_units_mean = sfo_year_data['housing_units'].mean()
    housing_units_per_year = housing_units_mean.hvplot.bar(ylim = (360000,390000))

    return housing_units_per_year
    
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    sfo_year_data = sfo_data.groupby('year')
    price_per_sqft = sfo_year_data['sale_price_sqr_foot','gross_rent'].mean()
    average_gross_rent = price_per_sqft['gross_rent'].hvplot()
    
    return average_gross_rent
    

def average_sales_price():
    """Average Sales Price Per Year."""
    
    sfo_year_data = sfo_data.groupby('year')
    price_per_sqft = sfo_year_data['sale_price_sqr_foot','gross_rent'].mean()
    average_sales_price = price_per_sqft['sale_price_sqr_foot'].hvplot()

    return average_sales_price

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    sfo_data_year_neighborhood = sfo_data.groupby(['year','neighborhood']).mean()
    average_price_by_neighborhood = neighborhood = sfo_data_year_neighborhood['sale_price_sqr_foot'].hvplot('year', groupby='neighborhood',xlabel='Year', ylabel='Price Per Sqft')

    return average_price_by_neighborhood

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    avg_neighborhood = sfo_data.groupby(['neighborhood']).mean()
    top_most_expensive_neighborhoods = avg_neighborhood['sale_price_sqr_foot'].sort_values(ascending=False).head(10)

    return top_most_expensive_neighborhoods

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # YOUR CODE HERE!

    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    avg_neighborhood = sfo_data.groupby(['neighborhood']).mean()
    parallel_coordinates = parallel_coordinates(avg_neighborhood, color ='gross_rent')

    
    return parallel_coordinates


def parallel_categories():
    """Parallel Categories Plot."""
    
    avg_neighborhood = sfo_data.groupby(['neighborhood']).mean()
    parallel_categories = px. parallel_categories(
        avg_neighborhood,
        dimensions = ['sale_price_sqr_foot', 'housing_units','gross_rent'],
        color = 'sale_price_sqr_foot',
        color_continuous_scale = px.colors.sequential.Inferno,
        labels = {'sale_price_sqr_foot':'Sale Price Per SQFT',
                 'housing_units':'Housing Units',
                 'gross_rent':'Gross Rent'
                 },
    )
    
    return parallel_categories


def neighborhood_map():
    """Neighborhood Map."""

    file_path = Path('neighborhoods_coordinates.csv')
    coordinates = pd.read_csv(file_path)

    avg_neighborhood = sfo_data.groupby(['neighborhood']).mean()
    merged_data = pd.merge(coordinates, avg_neighborhood, left_on= 'Neighborhood', right_on='neighborhood')
    neighborhood_map = px.scatter_mapbox(
        merged_data,
        lat= 'Lat',
        lon= 'Lon',
        size= 'housing_units',
        color= 'gross_rent',
        title= 'Average Rent in SFO',
    )
    
    return neighborhood_map


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [25]:
# Create a Title for the Dashboard
plot_column = pn.Column(
    "## Overview of SFO sales and rent market", 
    housing_units_per_year(), 
    average_gross_rent(),
    average_sales_price(),
    average_price_by_neighborhood(),
)

parallel_column = pn.Column(
    "## How variables in the SFO Market Interact",
    parallel_categories(),
)

map_column = pn.Column(
    '## Interactive Map of SFO Neighborhoods',
    neighborhood_map()
)

# Create a tab layout for the dashboard
panel = pn.Tabs(
    ('Plot', plot_column),
    ('Parallel', parallel_column),
    ('Map', map_column)
)

# Create the dashboard
panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
        [4] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [1] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)

## Serve the Panel Dashboard

In [26]:
# Serve the# dashboard
panel

Tabs(active=2)
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
        [4] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [1] Column
        [0] Markdown(str)
        [1] Plotly(Figure, relayout_data={'autosize': True})
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure, relayout_data={'autosize': True})

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()